In [40]:
import pandas as pd

In [42]:
df = pd.read_csv("../data/business_data.csv")
forecast = pd.read_csv("../data/forecast.csv")
production = pd.read_csv("../data/production_plan.csv")

In [44]:
price_data = df.groupby('Product')['Price'].mean().reset_index()
price_data.columns = ['Product','Avg_Price']

In [46]:
cost_data = df.groupby('Product')['Unit_Cost'].mean().reset_index()
cost_data.columns = ['Product','Avg_Unit_Cost']

In [48]:
forecast_demand = forecast.groupby('Product')['Forecasted_Demand'].sum().reset_index()
forecast_demand.columns = ['Product','Forecast_Demand']

In [50]:
profit_df = production.merge(price_data, on='Product')
profit_df = profit_df.merge(cost_data, on='Product')
profit_df = profit_df.merge(forecast_demand, on='Product')
profit_df.head()

,Product,Optimal_Production,Avg_Price,Avg_Unit_Cost,Forecast_Demand
0,Apples,342.50304,13.971358,6.287111,342.503036
1,Bananas,327.75597,14.561424,6.552641,327.755972
2,Bread,373.23352,14.706208,6.617794,373.233522
3,Carrots,281.85006,12.834174,5.775378,281.850062
4,Cereal,348.75089,14.101787,6.345804,348.750893


In [52]:
profit_df['Revenue'] = profit_df['Optimal_Production'] * profit_df['Avg_Price']
profit_df['Production_Cost'] = profit_df['Optimal_Production'] * profit_df['Avg_Unit_Cost']

In [54]:
holding = df.groupby('Product')['Holding_Cost'].mean().reset_index()
shortage = df.groupby('Product')['Shortage_Cost'].mean().reset_index()

profit_df = profit_df.merge(holding, on='Product')
profit_df = profit_df.merge(shortage, on='Product')

profit_df['Holding_Total'] = profit_df['Holding_Cost'] * profit_df['Optimal_Production']
profit_df['Shortage_Total'] = profit_df.apply(
    lambda row: row['Shortage_Cost'] * max(0, row['Forecast_Demand'] - row['Optimal_Production']),
    axis=1)

In [56]:
profit_df['Profit'] = (
    profit_df['Revenue']
    - profit_df['Production_Cost']
    - profit_df['Holding_Total']
    - profit_df['Shortage_Total']
)

profit_df[['Product','Profit']].head()

,Product,Profit
0,Apples,2588.810757
1,Bananas,2581.973178
2,Bread,2969.467796
3,Carrots,1956.966175
4,Cereal,2660.643861


In [58]:
profit_df.to_csv("../data/profit_results.csv", index=False)